In [ ]:
# Full:
plot_between('2013-01-01 00:00:00', '2014-09-30 23:59:59')

In [ ]:
# plot_between('2013-01-01 00:00:01','2013-03-31 23:59:59')

# data_combined['2013-01-01 00:00:01':'2013-03-31 23:59:59']

# plot_between('2013-01-01 00:00:00', '2014-08-31 23:59:59')
# Start
data_combined['2013-01-01 00:00:01':'2013-01-31 23:59:59']

In [ ]:
# Sum stats
print("From 2013-01-02 18:59:38 through 2014-08-22 10:21:32:\n")
print(data_combined['2013-01-02 18:59:38':'2014-08-22 10:21:32'].groupby('source', dropna=False)['raw'].agg(['count', 'mean','std', 'min', 'max']).dropna())

In [ ]:
plot_between('2013-01-02 18:59:38', '2014-08-22 10:21:32')
data_combined['2014-08-01 00:00:00':'2014-08-31 23:59:59']

In [ ]:
# data_combined['2013-01-01 00:00:01':'2013-01-31 23:59:59']
# data_combined['2013-01-01 00:00:01':'2013-01-31 23:59:59'].groupby('source', dropna=False)['raw'].agg(['count', 'mean','std', 'min', 'max']).dropna()
# data_combined['2013-01-02 18:59:38':'2014-08-22 10:21:32'].groupby('source', dropna=False)['raw'].agg(['count', 'mean','std', 'min', 'max']).dropna()
# print(data_combined['2013-01-01 00:00:01':'2014-09-30 23:59:59'].groupby('source', dropna=False)['raw'].agg(['count', 'mean','std', 'min', 'max']).dropna())
plot_between('2013-01-02 18:59:38', '2014-08-22 10:21:32')


In [ ]:
# checking old gap
data_all_combined[data_all_combined['source'] == "ISCO"][date_gap_start:date_gap_end]

In [ ]:
# data_start = data_all_combined[data_all_combined['source']=="ISCO"]["2013-01-01 00:00:00":"2013-03-15 00:00:00"]
data_start = data_all_combined[data_all_combined['source']!="CHART"]["2013-01-01 00:00:00":"2013-01-30 23:59:59"]


fig, ax = plt.subplots(figsize=(10, 6))
plt.axhline(y=0, color ='grey', linestyle = ':')
# Plot the rain as a bar chart with a multiplier for visibility
# ax.vlines(data_start.index, ymin=0, ymax=data_start['ra']*3, color = 'blue', label = "Rain (x3)")
ax.plot(data_start.index, data_start['level'], color = 'red', label = "Adjusted", linestyle='none', marker='.')
ax.plot(data_start.index, data_start['raw'], color = 'green', label = "Raw", linestyle='none', marker='.')
# ax.plot(data_start.index, data_start['level'], color = 'red', label = "Adjusted")
# ax.plot(data_start.index, data_start['raw'], color = 'green', label = "Raw")
# # Include calibration points unless otherwise specified or unless there are none in the subset
# if include_calibration == True and not data_subset_cal.empty:
#     ax.plot(data_subset_cal.index, data_subset_cal['weir_level'], linestyle='none', marker='x', color='red', label = "Calibration")

# Plot labels
ax.set_xlabel("Date (YYYY-MM-DD)")
ax.set_ylabel("Level (mm)")
# ax.set_title('Simple Time Series Plot')
# ax.set_title("Runoff time series from " + input_date_start + " through " + input_date_end)
# ax.set_ylim(bottom=0) 
# ax.grid(True)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
# Reverse the order of the legend
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], loc='upper right')
# plt.legend(loc = 'upper right')
plt.show()

del fig, ax, data_start, handles, labels

### CHART

Only non-CHART values will be used for making the model.
Prior to removing them, other missing values must also be dealt with, as they may relate to gaps within CHART-reliant ranges.

In [ ]:
# Modifying 
# Backup
data_combined_backup = data_combined.copy()
# Create a column which will forward fill the source--i.e., fill NAs with the most recent value reported in 'source'
data_combined_backup['source_ffill'] = data_combined_backup['source'].ffill()
# Create a column which will back fill the source--i.e., fill NAs with the next value reported in 'source'
data_combined_backup['source_bfill'] = data_combined_backup['source'].bfill()

# Filtering to remove CHART values and gap fills that rely on CHART values
data_combined_backup_1 = data_combined_backup[
    # Remove CHART values
    (data_combined_backup["source"] != "CHART") &
    # and
    (
        # selecting values with NA sources where the most recent & next sources are the same
        (data_combined_backup["source_ffill"] == data_combined_backup["source_bfill"]) |
        # or
        # selecting if there is no next value to fill
        # (i.e., the source remains the same into the present)
        (data_combined_backup["source_bfill"].isnull())
    ) &
    # and
    (((data_combined_backup["source_ffill"] != "CHART")) &
    ((data_combined_backup["source_bfill"] != "CHART")))
    # # remove NA values where the most recent source was CHART
    # ((data_combined_backup["source_ffill"] != "CHART") & data_combined_backup['source'].isnull()) &
    # # and
    # # remove NA-source values where the next source is CHART
    # ((data_combined_backup["source_bfill"] != "CHART") & data_combined_backup['source'].isnull())
    ]

data_combined_backup_2 = data_combined_backup[
    (data_combined_backup['source'] != "CHART") &
    (data_combined_backup['source_ffill'] != "CHART")
    &
    (data_combined_backup['source_bfill'] != "CHART")
]

print(len(data_combined_backup_1), "vs", len(data_combined_backup_2))
print(data_combined_backup_2[~data_combined_backup_2.index.isin(data_combined_backup_1.index)])


# mismatch: 2018-08-31 10:00:00 

data_combined_backup_2["2018-08-31 09:00:00":"2018-09-30 23:59:59"]

del data_combined_backup_1, data_combined_backup_2, data_combined_backup

# data_combined_backup[data_combined_backup['source'].isnull()]
# 3,555,834
# 3,134,121 
# 3,111,793

In [ ]:
# del data_combined_backup_1, data_combined_backup_2, data_combined_backup
data_combined[(data_combined['chk_fail'].str.contains("Gap Fill")) & (data_combined['source'].notnull())]

In [ ]:
# Modifying 
# Backup
# data_combined_mod = data_combined.copy()
data_combined_mod = data_all_combined.copy()
data_combined_mod = data_combined_mod[date_weir_start:date_weir_end]

# Create a column which will forward fill the source--i.e., fill NAs with the most recent value reported in 'source'
data_combined_mod['source_ffill'] = data_combined_mod['source'].ffill()
# Create a column which will back fill the source--i.e., fill NAs with the next value reported in 'source'
data_combined_mod['source_bfill'] = data_combined_mod['source'].bfill()

# Filtering to remove CHART values and gap fills that rely on CHART values
data_combined_mod = data_combined_mod[
    # Remove CHART values
    (data_combined_mod["source"] != "CHART") &
    # and
    (
        # selecting values with NA sources where the most recent & next sources are the same
        (data_combined_mod["source_ffill"] == data_combined_mod["source_bfill"]) |
        # or
        # selecting if there is no next value to fill
        # (i.e., the source remains the same into the present)
        (data_combined_mod["source_bfill"].isnull())
    ) &
    # and
    # remove NA values where the most recent source was CHART
    (data_combined_mod["source_ffill"] != "CHART") &
    # and
    # remove NA-source values where the next source is CHART
    (data_combined_mod["source_bfill"] != "CHART")
    ]

# data_combined_mod

# 3,555,834 with removing gap
# 3,557,018 without removing gap


data_combined_mod_2 = data_combined.copy()
# Create a column which will forward fill the source--i.e., fill NAs with the most recent value reported in 'source'
data_combined_mod_2['source_ffill'] = data_combined_mod_2['source'].ffill()
# Create a column which will back fill the source--i.e., fill NAs with the next value reported in 'source'
data_combined_mod_2['source_bfill'] = data_combined_mod_2['source'].bfill()

# Filtering to remove CHART values and gap fills that rely on CHART values
data_combined_mod_2 = data_combined_mod_2[
    # Remove CHART values
    (data_combined_mod_2["source"] != "CHART") &
    # and
    (
        # selecting values with NA sources where the most recent & next sources are the same
        (data_combined_mod_2["source_ffill"] == data_combined_mod_2["source_bfill"]) |
        # or
        # selecting if there is no next value to fill
        # (i.e., the source remains the same into the present)
        (data_combined_mod_2["source_bfill"].isnull())
    ) &
    # and
    # remove NA values where the most recent source was CHART
    (data_combined_mod_2["source_ffill"] != "CHART") &
    # and
    # remove NA-source values where the next source is CHART
    (data_combined_mod_2["source_bfill"] != "CHART")
    ]

# data_combined_mod

# data_combined_mod.index.symmetric_difference(data_combined_mod_2.index)
# Merge DataFrames on common columns, indicating source
# merged_df = data_combined_mod.merge(data_combined_mod_2, on=['ID', 'Name'], how='outer', indicator=True)

# Filter for rows not present in both DataFrames
# diff_rows = merged_df[merged_df['_merge'] != 'both']


In [ ]:
# data_combined_mod.index.symmetric_difference(data_combined_mod_2.index)
# 3,555,834 with removing gap
# 3,557,018 without removing gap
merged_df = pd.merge(data_combined_mod, data_combined_mod_2, left_index=True, right_index=True, how='left', indicator=True)
# merged_df[merged_df["_merge"] == "right_only"]
# merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge', 'value_right'])

In [ ]:
# print(abs(3555834 - 3557018))
# len(merged_df[merged_df["_merge"] == "left_only"])
flag_index = merged_df[merged_df["_merge"] == "left_only"].index
# data_all_combined[date_gap_start:date_gap_end]
data_flagged = data_all_combined.copy()

data_flagged["flag"] = False

data_flagged.loc[data_flagged.index.isin(flag_index), "flag"]  = True

# data_flagged[date_gap_start:date_gap_end]
# data_flagged[date_gap_end:]
data_flagged[data_flagged["flag"]==True]

# merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge', 'value_right'])

In [ ]:
data_flagged_diff = data_flagged[data_flagged["flag"]==True]
full_date_range = pd.date_range(start = data_flagged_diff.index[0], end = data_flagged_diff.index[-1], freq="5min")
data_flagged_diff = data_flagged_diff.reindex(full_date_range)
# data_flagged_diff

fig, ax = plt.subplots(figsize=(10, 6))
## Line for 0
plt.axhline(y=0, color = "grey", linestyle = ":")
# Mean
ax.plot(data_flagged_diff.index, data_flagged_diff['level'], color = 'green')
# ax.scatter(data_flagged_diff.index, data_flagged_diff['level'], color = 'green')
# ax.plot(data_sumstats.index, data_sumstats['mean'], color = 'green', label = "Mean")
# Ribbon for standard deviation
# ax.fill_between(data_mixed_filtered_sumstats.index, data_mixed_filtered_sumstats['mean']-data_mixed_filtered_sumstats['std'], data_mixed_filtered_sumstats['mean']+data_mixed_filtered_sumstats['std'], color = 'aquamarine', label = "std")
ax.set_xlabel("Year")
ax.set_ylabel("Level (mm)")
ax.set_title("Average raw values every 1mo")
# ax.set_ylim(bottom = 0)
# ax.set_xlim(left = dt.date(1989, 1, 1), right = dt.date(2026, 1, 1))
# ax.xaxis.set_major_locator(mdates.YearLocator(month = 1)) # Show ticks at start of year
plt.xticks(rotation = 90)
plt.tight_layout()
plt.grid(axis = 'x', which = 'major')
# plt.legend(loc = 'upper right')
# Truncate plot
# ax.set_ylim(bottom = 0, top = 250)

plt.show()

del fig, ax, data_flagged_diff

### Missing

In [ ]:
# data_combined['source'].value_counts(dropna=False)
# data_combined['chk_note'].value_counts(dropna=False)
data_combined.groupby(['chk_note', 'source'],dropna=False).size()
# data_combined[data_combined['source'].isnull()]

Other gaps of missing values occur and should be addressed.
These can be identified by the `chk_note` of 'missing' with a `raw` values of -999.0.

A `chk_note` of 'missing' differs from instances of where a `chk_fail` is a 'Gap Fill'.

In [ ]:
data_combined.groupby(['chk_note', 'chk_fail'],dropna=False).size()

In [ ]:
# data_combined[(data_combined['chk_note'] == 'missing') & (data_combined['raw'] != -999.0)]
# data_combined[data_combined['source'].isnull()]
# data_combined['1997-01-01 00:00:00':'1997-01-01 23:59:59']
# data_combined[data_combined['source']=='CHART']
data_combined[(data_combined['chk_note'] == 'missing') & (data_combined['source'] == 'CHART')]
# data_combined[(data_combined['chk_note'] == 'missing') & (data_combined['level'] != 0)]
# data_combined['1993-03-08 00:00:00':'1993-03-08 23:59:59']

In [ ]:
# data_combined[data_combined['source'] == 'CHART']
data_chart = data_combined[data_combined['source'] == 'CHART'].resample('D').size().rename("n")
data_chart = data_chart.replace(0, np.nan)
# data_chart = data_chart[data_chart > 0]
# data_chart[data_chart["n"] > 0]
# data_chart.columns

# data_chart[data_chart['0'] != 0]
# data_chart.rename(columns={0: "blah"})

plt.figure(figsize=(20, 6))
plt.axhline(y=0, color ='grey', linestyle = ':')
# plt.scatter(data_chart.index, data_chart, marker="x")
plt.plot(data_chart.index, data_chart)
plt.show()


In [ ]:
# # data_combined.index isin data_chart.dropna().index
# # data_chart.dropna()
# data_tally = data_combined["source"]
# data_tally = pd.DataFrame(data_tally)
# # data_tally['B'] = (~(data_combined["source"] == "CHART")).cumsum()
# # data_tally["2000-01-01 00:00:00"]
# # data_tally
# # data_tally['B'] = (~df['A']).cumsum()
# # data_tally['A'].sum()
# # data_tally['C'] = data_tally.duplicated(['source', 'B'], keep='first')
# # data_tally['C'].sum()

# data_tally['A'] = (data_tally["source"]=="CHART")
# # data_tally['B'] = (~data_tally['A']).cumsum()
# # data_tally['C'] = data_tally.duplicated(['A', 'B'], keep='first')
# # data_tally['C'].sum()
# # data_tally


# (data_tally['A']&data_tally['A'].shift(fill_value=False)).sum()

In [ ]:
# data_chart_days["chk_fail"].str.contains("Gap Fill").dropna()
# data_chart_days[(data_chart_days["chk_fail"] == "Gap Fill")]

In [ ]:
# data_chart.dropna().index
# data_combined.index.date
# data_chart.dropna().index.date
# data_combined.index.date
dates_to_filter = data_chart.dropna().index.date
# dates_to_filter.date
data_chart_days = data_combined[data_combined.index.floor('D').isin(dates_to_filter)]
# data_chart_days
# data_chart_days["source"].groupby(data_chart_days["source"].ne(data_chart_days["source"].shift()).cumsum()).cumcount()
# data_chart_days["tally"] = data_chart_days["source"].ne(data_chart_days["source"].shift()).cumsum()
# condition_gap = (data_chart_days["raw"] == -999.0) & (data_chart_days["chk_fail"] == "Gap Fill") & (data_chart_days["source"].isnull())
# data_chart_days[condition_gap]

# condition_gap = (data_chart_days["raw"] == -999.0) & (data_chart_days["chk_fail"].str.contains("Gap Fill")) & (data_chart_days["source"].isnull())
# # data_chart_days["source_mod"] = data_chart_days["source_mod"].astype(str)
# # data_chart_days.loc[condition_gap, "source_mod"] = "NA"

# data_chart_days["source_mod"] = data_chart_days["source"]
# data_chart_days.loc[condition_gap, "source_mod"] = "CHART"
# data_chart_days.loc[(data_chart_days["chk_note"] == "missing"), "source_mod"] = "CHART"

##
data_chart_days["source_mod"] = data_chart_days["source"]
data_chart_days["source_mod"] = data_chart_days["source_mod"].ffill()
##

# data_chart_days["source_mod"] = data_chart_days[]

# data_chart_days = data_chart_days.drop(['chk_note', 'chk_fail', 'comment'],axis=1)

# data_chart_days.loc[(data_chart_days["source_mod"].isnull())]
data_chart_days["group_tally"] = data_chart_days["source_mod"].ne(data_chart_days["source_mod"].shift()).cumsum(skipna=False)
data_chart_days["tally"] = data_chart_days["source_mod"].groupby(data_chart_days["source_mod"].ne(data_chart_days["source_mod"].shift()).cumsum(skipna=False)).cumcount()

data_chart_days["tally"] = data_chart_days["tally"]+1
# data_chart_days

# data_chart_days.groupby('group_tally')['tally'].max()
data_grouped_tally = data_chart_days.groupby('group_tally')['tally'].max()
# data_grouped_tally[data_grouped_tally == 1]
# data_grouped_tally == 1

data_chart_days[data_chart_days["group_tally"].isin(data_grouped_tally[data_grouped_tally == 1].index)]
# 1996-11-12 01:25:00, 2007-10-01 09:46:00
# data_chart_days["2007-10-01 00:00:00":"2007-10-01 23:59:59"]
# data_chart_days["group"]
# data_chart_days.loc[data_grouped_tally["tally"], "tally"]

# data_chart_days
# data_grouped_tally[data_grouped_tally == 1].index
# data_chart_days["tally"] == 1

# data_chart_days = data_chart_days.drop(['chk_note', 'chk_fail', 'comment'],axis=1)
# data_chart_days["group_tally"] = data_chart_days["source"].ne(data_chart_days["source"].shift()).cumsum()
# data_chart_days["tally"] = data_chart_days["source"].groupby(data_chart_days["source"].ne(data_chart_days["source"].shift()).cumsum()).cumcount()
# data_chart_days

# plt.figure(figsize=(20, 6))
# plt.axhline(y=0, color ='grey', linestyle = ':')
# # plt.scatter(data_chart.index, data_chart, marker="x")
# for source in data_chart_days["source"].unique():
#     # plt.scatter(data_chart_days.index, (data_chart_days["source"]==source))
#     plt.scatter(range(len(data_chart_days)), (data_chart_days["source"]==source), label=source, s=0.25)
#     # plt.scatter(range(len(data_chart_days)), data_chart_days[data_chart_days["source"]==source]["level"])
# # plt.scatter(range(len(data_chart_days)), data_chart_days['raw'], hue="source")
# plt.legend()
# plt.show()
# # data_combined[data_combined.index.isin(data_chart.dropna().index)]
# # data_combined.loc[data_chart.dropna().index.date]


In [ ]:
# data_combined[(data_combined['chk_note']=="missing") & (data_combined['level']!= 0)]
# data_combined[(data_combined['chk_note']!="missing") & (data_combined['raw']== -999.0)]
# data_combined[(data_combined['chk_note']=="missing") & (data_combined['raw']== -999.0)]
# data_combined[(data_combined['chk_fail'].str.contains("Gap Fill") == False) & (data_combined['chk_note']!="missing") & (data_combined['raw']== -999.0)]
# data_combined[(data_combined['chk_fail'].str.contains("Gap Fill") == False) & (data_combined['raw']== -999.0)]
# data_combined[(data_combined['raw']== -999.0) & (data_combined['level']!=0)]
# data_combined[(data_combined['chk_note']=="missing") & (data_combined['raw']!= -999.0)]
# print(len(data_combined[(data_combined['chk_note']!="missing") & (data_combined['source']!= "CHART")]))
# print(len(data_combined[(data_combined['chk_note']!= "missing")]))
# print(len(data_combined[(data_combined['source']== "CHART")]))

# print(len(data_combined[(data_combined['chk_note']!= "missing")]) - len(data_combined[(data_combined['source']== "CHART")]))

# data_combined[(data_combined['chk_note']!="missing") & (data_combined['source']== "CHART")]
# dates_gaps = set(data_combined[(data_combined['chk_fail'].str.contains("Gap Fill", na=False))].index.date)

# data_combined[data_combined.index.floor('D').isin(dates_gaps)]

data_mixed = data_combined.copy()
data_mixed["source_ffill"] = data_mixed["source"].ffill()
data_mixed["source_bfill"] = data_mixed["source"].bfill()
# data_mixed[(data_mixed['chk_fail'].str.contains("Gap Fill")) | (data_mixed['chk_note']=="missing") | (data_mixed['raw']== -999.0)]

# data_mixed[(data_mixed['chk_fail'].str.contains("Gap Fill")) & (data_mixed['source'].isnull()) & ((data_mixed['source_ffill'] != "CHART") | (data_mixed['source_bfill'] != "CHART"))]
# 07-09
# data_mixed["1997-07-17 00:00:00":"1997-07-21 23:59:59"]
# data_mixed["1997-03-19 00:00:00":"1997-03-19 23:59:59"]
# print(len(data_mixed[(data_mixed['source'] == "CHART")]),
#     len(data_mixed[(data_mixed['source_bfill'] == "CHART") & (data_mixed['chk_fail'].str.contains("Gap Fill", na=False))])
# )

data_mixed[(data_mixed["source_bfill"] != data_mixed["source_ffill"]) & ((data_mixed["source_bfill"] != "CHART") | (data_mixed["source_ffill"] != "CHART"))]
# data_mixed["2002-12-27 07:00:00":"2002-12-27 10:00:00"]

# data_mixed[data_mixed['chk_']]
# data_combined[(data_combined['chk_fail'].str.contains("Gap Fill", na=False))]
# len(data_combined[(data_combined['chk_note']!="missing")]) + len(data_combined[(data_combined['source']!= "CHART")])

# set(data_combined[data_combined['raw'] == -999.0].index.date)

In [ ]:

data_mixed_filtered = data_mixed[
    (data_mixed["source"] != "CHART") &
    ((data_mixed["source_ffill"] == data_mixed["source_bfill"]) | data_mixed["source_bfill"].isnull()) &
    (data_mixed["source_ffill"] != "CHART") &
    (data_mixed["source_bfill"] != "CHART")
    ]#["2002-12-26 00:00:00":"2002-12-26 23:59:59"]

# data_mixed_filtered[~data_mixed_filtered["chk_fail"].isnull()]["chk_fail"].unique()
# data_mixed_filtered[data_mixed_filtered["chk_fail"].str.contains("Gap Fill", na=False)]["chk_note"].unique()
# data_mixed_filtered[data_mixed_filtered["raw"] == -999.0]

# data_mixed_filtered[data_mixed_filtered["chk_note"] == "missing"]
# data_mixed_filtered[(data_mixed_filtered["raw"] == -999.0)]["chk_fail"].unique()
# data_mixed_filtered[(data_mixed_filtered["raw"] == -999.0) & (data_mixed_filtered["level"] != 0)]#["chk_fail"].unique()
data_mixed_filtered[(data_mixed_filtered["raw"] < 0) & (data_mixed_filtered["raw"] != -999.0) & (data_mixed_filtered["level"] > 0)]
# data_mixed_filtered[(data_mixed_filtered["raw"] == -999.0) & (data_mixed_filtered["chk_fail"].isnull())]


In [ ]:
data_combined[data_combined["chk_fail"].str.contains(",", na=False)]

In [ ]:
data_mixed_filtered_sumstats = data_mixed_filtered['raw'].dropna().resample('1ME').agg(['count', 'mean','std', 'min', 'max'])

# data_mixed[
#     # (data_mixed["source"] == "CHART") |
#     ((data_mixed["source_ffill"] != data_mixed["source_bfill"]))
#     # data_mixed["source"].isnull()
#     ]

# data_mixed[((data_mixed["source_ffill"] != data_mixed["source_bfill"]))]
# data_mixed["2002-12-26 00:00:00":"2002-12-26 23:59:59"]

fig, ax = plt.subplots(figsize=(20, 3))
## Line for 0
plt.axhline(y=0, color = "grey", linestyle = ":")
# Mean
ax.plot(data_mixed_filtered_sumstats.index, data_mixed_filtered_sumstats['mean'], color = 'green')
# ax.plot(data_sumstats.index, data_sumstats['mean'], color = 'green', label = "Mean")
# Ribbon for standard deviation
# ax.fill_between(data_mixed_filtered_sumstats.index, data_mixed_filtered_sumstats['mean']-data_mixed_filtered_sumstats['std'], data_mixed_filtered_sumstats['mean']+data_mixed_filtered_sumstats['std'], color = 'aquamarine', label = "std")
ax.set_xlabel("Year")
ax.set_ylabel("Level (mm)")
ax.set_title("Average raw values every 1mo")
# ax.set_ylim(bottom = 0)
ax.set_xlim(left = dt.date(1989, 1, 1), right = dt.date(2026, 1, 1))
ax.xaxis.set_major_locator(mdates.YearLocator(month = 1)) # Show ticks at start of year
plt.xticks(rotation = 90)
plt.tight_layout()
plt.grid(axis = 'x', which = 'major')
# plt.legend(loc = 'upper right')
# Truncate plot
# ax.set_ylim(bottom = 0, top = 250)

plt.show()

del fig, ax

### Exploration

In [ ]:
data_nochart = data_combined.copy()
data_nochart['source_ffill'] = data_nochart['source'].ffill()
data_nochart['source_bfill'] = data_nochart['source'].bfill()
data_nochart = data_nochart[
    (data_nochart['source'] != "CHART") &
    (data_nochart['source_ffill'] != "CHART") &
    (data_nochart['source_bfill'] != "CHART")
]

# data_mixed_filtered_sumstats = data_mixed_filtered['raw'].dropna().resample('1ME').agg(['count', 'mean','std', 'min', 'max'])
data_mixed_filtered_sumstats = data_nochart['raw'].dropna().resample('1W').agg(['count', 'mean','std', 'min', 'max'])
# data_mixed_filtered_sumstats = data_nochart['raw'].dropna().resample('1ME').agg(['count', 'mean','std', 'min', 'max'])

# data_mixed[
#     # (data_mixed["source"] == "CHART") |
#     ((data_mixed["source_ffill"] != data_mixed["source_bfill"]))
#     # data_mixed["source"].isnull()
#     ]

# data_mixed[((data_mixed["source_ffill"] != data_mixed["source_bfill"]))]
# data_mixed["2002-12-26 00:00:00":"2002-12-26 23:59:59"]

fig, ax = plt.subplots(figsize=(20, 3))
## Line for 0
plt.axhline(y=0, color = "grey", linestyle = ":")
# Mean
ax.plot(data_mixed_filtered_sumstats.index, data_mixed_filtered_sumstats['mean'], color = 'green')
# ax.plot(data_sumstats.index, data_sumstats['mean'], color = 'green', label = "Mean")
# Ribbon for standard deviation
# ax.fill_between(data_mixed_filtered_sumstats.index, data_mixed_filtered_sumstats['mean']-data_mixed_filtered_sumstats['std'], data_mixed_filtered_sumstats['mean']+data_mixed_filtered_sumstats['std'], color = 'aquamarine', label = "std")
ax.set_xlabel("Year")
ax.set_ylabel("Level (mm)")
ax.set_title("Average raw values every 1mo")
# ax.set_ylim(bottom = 0)
ax.set_xlim(left = dt.date(1989, 1, 1), right = dt.date(2026, 1, 1))
ax.xaxis.set_major_locator(mdates.YearLocator(month = 1)) # Show ticks at start of year
plt.xticks(rotation = 90)
plt.tight_layout()
plt.grid(axis = 'x', which = 'major')
# plt.legend(loc = 'upper right')
# Truncate plot
# ax.set_ylim(bottom = 0, top = 250)

plt.show()

del fig, ax

In [ ]:
# data_nochart[data_nochart['raw'] < 0]
data_nochart[data_nochart['raw'] < 0]['raw'].dropna().resample('1ME').agg(['count', 'mean','std', 'min', 'max']).dropna()